# Librerias

In [3]:
import numpy as np
import matplotlib.pyplot as plt

#Librosa
from librosa import lpc
from librosa.effects import preemphasis
from librosa import resample
from librosa.util import peak_pick

# Funciones

In [ ]:
NUMBER_OF_SLOTS = 4
estimates = np.zeros(NUMBER_OF_SLOTS)

def Peaks2Formants(peaks):
    slots = np.zeros(NUMBER_OF_SLOTS) #Slots where potential formants are placed
    used_indx = []
    #STEP 1 - fill slots based on closest distance to estimates
    for i in range(0, NUMBER_OF_SLOTS):
        indx = np.argmin( abs(peaks-estimates[i]) )
        slots[i] = peaks[indx] 
        used_indx.append(indx)
        
    #STEP 2 - remove duplicates
    u, indices = np.unique(slots, return_index=True)
    for i in range(0, NUMBER_OF_SLOTS):
        if i not in  indices:
            slots[i] = 0
    
    #STEP 3 - deal with unused peaks
    unused_peaks = np.delete(peaks,used_indx)
    for peak in unused_peaks:
        k = np.argmin(peaks - peak)
        if slot[k] == 0: # Si el slot con el mismo numero esta vacio lo asigno ahi
            slot[k] = peak
        elif peak > 0.5*slot[k]:
            if k < NUMBER_OF_SLOTS:
                if slot[k+1] == 0: 
                    slot[k+1] = slot[k] #Muevo el pico del slot uno para adelante
                    slot[k] = peak #Asigno el pico sin usar al slot con su numero
                else:
                    if k > 0:
                        if slot[k-1] == 0: 
                            slot[k-1] = slot[k] #Muevo el pico del slot uno para atras
                            slot[k] = peak #Asigno el pico sin usar al slot con su numero
    estimates = slots
    return slots
    
def GetFormants(buffer, fs, energy_th=100, N=256, new_fs=1e4, lpc_order=14):
    if( buffer.std < energy_th )
        return None #Descarto el frame si es silencio
    hp_filtered = preemphasis(buffer, coef=1)
    downsampled = resample(hp_filtered, fs, new_fs)
    coefs = lpc(downsampled, lpc_order) #Revisar si son coeficientes del filtro de error o del predictivo
    spectrum = 1/abs(np.fft.rfft([1, -coefs],n=N)) 
    peaks = peak_pick(spectrum, pre_max=3, post_max=3, pre_avg=3, post_avg=3, delta=0, wait=0)
    #Aca falta meter la parte de obtener las primeras
    return peaks

# Cargo señal de audio y estimo formantes